In [1]:
import pandas as pd
import numpy as np
import requests

In [3]:

with open("golf_API_key.txt", "r") as file:
    api_key = file.read().strip()
    
base_url = "https://live-golf-data.p.rapidapi.com/leaderboard"

params = {"orgId":"1","tournId":"519","year":"2024"}

headers =  {
	"x-rapidapi-key": api_key,
	"x-rapidapi-host": "live-golf-data.p.rapidapi.com"
}

response = requests.get(base_url, headers=headers, params=params)

print(response.json())

{'_id': {'$oid': '66af2b54ff0cdf8031b02ed2'}, 'orgId': '1', 'year': '2024', 'tournId': '519', 'status': 'Official', 'roundId': {'$numberInt': '4'}, 'roundStatus': 'Official', 'lastUpdated': {'$date': {'$numberLong': '1722788353233'}}, 'timestamp': {'$date': {'$numberLong': '1722788353233'}}, 'cutLines': [], 'leaderboardRows': [{'lastName': 'Scheffler', 'firstName': 'Scottie', 'playerId': '46046', 'isAmateur': False, 'courseId': '250', 'status': 'complete', 'position': '1', 'total': '-19', 'currentRoundScore': '-9', 'totalStrokesFromCompletedRounds': '265', 'currentHole': {'$numberInt': '18'}, 'startingHole': {'$numberInt': '1'}, 'roundComplete': True, 'rounds': [{'scoreToPar': '-4', 'roundId': {'$numberInt': '1'}, 'strokes': {'$numberInt': '67'}, 'courseId': '250', 'courseName': 'Le Golf National'}, {'scoreToPar': '-2', 'roundId': {'$numberInt': '2'}, 'strokes': {'$numberInt': '69'}, 'courseId': '250', 'courseName': 'Le Golf National'}, {'scoreToPar': '-4', 'roundId': {'$numberInt': '3

In [34]:
data = response.json()
scoreboard_list = []

for score in data['leaderboardRows']:
    rounds = score.get('rounds', [])
    first_round_score = rounds[0].get('strokes').get('$numberInt') if len(rounds) > 0 else None
    second_round_score = rounds[1].get('strokes').get('$numberInt') if len(rounds) > 1 else None
    third_round_score = rounds[2].get('strokes').get('$numberInt') if len(rounds) > 2 else None
    forth_round_score = rounds[3].get('strokes').get('$numberInt') if len(rounds) > 3 else None

    score_info = {
        'First Name': score.get('firstName'),
        'Last Name' : score.get('lastName'),
        'Position' : score.get('position'),
        'Strokes Under' : score.get('total'),
        'Total Strokes' : score.get('totalStrokesFromCompletedRounds'),
        'First Round': first_round_score,
        'Second Round' : second_round_score,
        'Third Round' : third_round_score,
        'Forth Round' : forth_round_score,
        'Forth Round Tee Time' : score.get('teeTime'),
        'Amateur Status': score.get('isAmateur'),
        'Tournament Status' : score.get('status'),
        'ID' : score.get('playerId')
    }
    scoreboard_list.append(score_info)

scoreboard_df = pd.DataFrame(scoreboard_list)
scoreboard_df


,First Name,Last Name,Position,Strokes Under,Total Strokes,First Round,Second Round,Third Round,Forth Round,Forth Round Tee Time,Amateur Status,Tournament Status,ID
0,Scottie,Scheffler,1,-19,265,67,69,67,62,12:17pm,False,complete,46046
1,Tommy,Fleetwood,2,-18,266,67,64,69,66,12:39pm,False,complete,30911
2,Hideki,Matsuyama,3,-17,267,63,68,71,65,12:28pm,False,complete,32839
3,Victor,Perez,4,-16,268,70,67,68,63,11:55am,False,complete,47679
4,Rory,McIlroy,T5,-15,269,68,69,66,66,12:28pm,False,complete,28237
5,Jon,Rahm,T5,-15,269,67,66,66,70,12:39pm,False,complete,46970
6,Nicolai,Højgaard,7,-14,270,70,70,62,68,12:28pm,False,complete,52453
7,Tom,Kim,8,-13,271,66,68,69,68,12:17pm,False,complete,55182
8,Corey,Conners,T9,-12,272,68,69,69,66,11:44am,False,complete,39997
9,Jason,Day,T9,-12,272,69,68,67,68,12:06pm,False,complete,28089


In [36]:
round_list = []
data = response.json()

for player in data['leaderboardRows']:
    first_name = player.get('firstName')
    last_name = player.get('lastName')
    full_name = f"{first_name} {last_name}"
    player_id = player.get('playerId')

    for round in player.get('rounds', []):
        round_info = {
            'Name': full_name,
            'Player Id' : player_id,
            'Strokes Under' : round.get('scoreToPar'),
            'Round' : round.get('roundId', {}).get('$numberInt'),
            'Round Strokes': round.get('strokes', {}).get('$numberInt'),
        }
        round_list.append(round_info)

rounds_df = pd.DataFrame(round_list)
rounds_df

,Name,Player Id,Strokes Under,Round,Round Strokes
0,Scottie Scheffler,46046,-4,1,67
1,Scottie Scheffler,46046,-2,2,69
2,Scottie Scheffler,46046,-4,3,67
3,Scottie Scheffler,46046,-9,4,62
4,Tommy Fleetwood,30911,-4,1,67
...,...,...,...,...,...
233,Matt Fitzpatrick,40098,-7,2,64
234,Matt Fitzpatrick,40098,+10,3,81
235,Phachara Khongwatmai,47239,-1,1,70
236,Phachara Khongwatmai,47239,+4,2,75
